## Using GPT model


In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEl = "gpt-3.5-turbo"

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(api_key = OPENAI_API_KEY, model = MODEl)
model.invoke("Tell me a joke")


## Using LLama model


In [16]:
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = "llama2"

In [17]:
from langchain_community.llms import Ollama

model = Ollama(model = MODEL)
model.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😄"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("Tell me a joke")

'Why was the math book sad? Because it had too many problems! 😂'

### Simple RAG system


In [18]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("protocol.pdf")
pages = loader.load_and_split()

pages

[Document(page_content='CLINICAL STUDY PROTOCOL \n \n \nProtocol Title:  A Phase 1/2 Multicenter Study Evaluating the Safety and Efficacy of \nKTE -C19 in Subjects with Refractory Aggressiv e Non -Hodgkin \nLymphoma (ZUMA -1) \nProtocol Number:  KTE -C19-101 \nIND Number:  016278  \nEudraCT Number:  2015 -005007 -86  \nClinical Study Sponsor:  Kite Pharma, Inc.  \n2400 Broadway  \nSanta Monica, CA 90404  \nUnited States of America  \nKey Sponsor Contacts:  , Clinical Development  \nKite Pharma, Inc.  \n2 Roundwood Avenue  \nStockley Park  \nUxbridge, Middlesex  \nPhone:  \nEmail:  \n  Clinical Development  \nKite Pharma Inc.  \n2400 Broadway  \nSanta Monica, CA  90404  \nPhone:  \nEmail:   \n  Clinical Operations  \nKite Pharma, Inc.  \n2400 Broadway  \nSanta Monica, CA 90404  \nPhone:  \nEmail:  \nVersion:  1.0 (Amendment #  8) \nDate:  11 February  2019 \n \nConfidentiality Notice \nThis document contains proprietary and confidential information of Kite Pharma Inc., a wholly \nowned 

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the given context.

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
print (prompt.format(context="here is the context", question="here is the question"))



Answer the question based on the given context.

Context: here is the context

Question: here is the question




In [6]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\UDIT\anaconda3\envs\miimansa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
retriever = vectorstore.as_retriever()

retriever.invoke("Zuma 1")

[Document(page_content='Axicabtagene ciloleucel; KTE -C19 \nClinical Protocol: KTE-C l9-10 1 \nFigure 3. Phase 1 Dosing Cohort s and Regimen s \nYes* \n*May be explored : see Section 9.6 \nCONFIDENTIAL Page 27 of 118 Kite Phanna , Inc. \nFinal \nNo \n11 Februaiy 2019', metadata={'source': 'protocol.pdf', 'page': 26}),
 Document(page_content='Axicabtagene ciloleucel; KTE-C19 Kite Pharma, Inc. \nClinical Protocol: KTE-C19- 101 Final  \nCONFIDENTIAL Page 115 of 118  11 February 2019 \n 18. A\nPPENDICES  \nAppendix A  Revised IWG Response Criteria for Malignant Lymphoma  (Cheson et al, \n2007)  \nAppendix B  Monitoring of subjects after IP administration per country regulatory agencies:', metadata={'source': 'protocol.pdf', 'page': 114}),
 Document(page_content='Axicabtag ene ciloleuc el; KTE-C19 \nClinica l Protocol: KTE -Cl 9- 101 Kite Phanna , Inc. \nFinal \nhas been studied and tolerated in subjects with B cell malignancies (O\'Brien et al, 2001). Similar \ntotal doses of cyclophospham

In [8]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "How many phases is the zuma 1 study separated into?"})

'Based on the given context, the zuma 1 study is separated into 3 phases:\n\nPhase 1: This phase includes the screening period and the treatment period.\nPhase 2: This phase includes the pivotal study and the safety management study.\nPhase 3: This phase includes the long-term follow-up period.\n\nTherefore, the zuma 1 study is separated into 3 phases.'

In [10]:
Questions = [
    "How many phases is the zuma 1 study separated into ?",
    "How many cohorts are there in the zuma 1 study ?",
    "What periods in the zuma 1 study does each subject proceed through ?",
    "What is the primary objective of the Phase 1 zuma 1 study ?",
    "What are the secondary objectives of the Phase 1 study ?",
    "What is the primary objective of the Phase 2 pivotal study ?",
    "What are the secondary objectives of the Phase 2 pivotal study",
    "What is the primary objective of the phase 2 safety management study?",
    "What are the secondary objectives of the Phase 2 safety management study",
    "What is the primary endpoint of the phase 1 study",
    "What are the secondary endpoints of the phase 1 study",
    "What is the primary endpoint of the phase 2 pivotal study?",
    "What are the secondary endpoints of the phase 2 pivotal study",
    "What is the primary endpoint of the phase 2 safety management study",
    "What are the secondary endpoints of the phase 2 safety management studies",
    "How many subjects will be enrolled in the zuma 1study?",
    "How many subjects will be enrolled in the phase 1 zuma 1study",
    "How many subjects will be enrolled in the phase 2 pivotal study",
    "How many subjects will be enrolled in the cohort 1 of the phase 2 pivotal study",
    "How many subjects will be enrolled in the cohort 2 of the phase 2 pivotal study",
    "How many subjects will be enrolled in the phase 2 safety management study",
    "How many subjects will be enrolled in cohort 3 phase 2 safety management study",
    "How many subjects will be enrolled in cohort 4 phase 2 safety management study",
    "How many subjects will be enrolled in cohort 5 phase 2 safety management study",
    "How many subjects will be enrolled in cohort 6 phase 2 safety management study",
    "What is the investigational product in this study",
    "What does the investigational product treatment consist of",
    "What does axicabtagene ciloleucel treatment consist of",
    "When may bridging therapy be considered?",
    "When may debulking therapy be considered?",
    "Which subjects may be considered for debulking therapy?",
    "What conditioning chemotherapy is used before axicabtagene ciloleucel is administered?",
    "What is the dose of the conditioning chemotherapy?",
    "What is the conditioning chemotherapy regimen?",
    "What is the fludarabine dose used in conditioning chemotherapy?",
    "What is the cyclophosphamide dose used in conditioning chemotherapy?",
    "For how many days is the conditioning chemotherapy given?",
    "What treatment is given to subjects in cohort 3 of the phase 2 safety management study?",
    "What treatment is given to subjects in cohort 4 of the phase 2 safety management study?",
    "What conditioning chemotherapy is given to subjects in cohort 5 of the phase 2 safety management study?",
    "What treatment is given to subjects in cohort 5 of phase 2 safety management study?",
    "What conditioning chemotherapy is given to subjects in cohort 6 of the phase 2 safety management study?",
    "According to the schedule of assessments, what assessments or procedures will the subjects undergo at regular time periods?",
    "What procedures will the subjects undergo at baseline",
    "Which subjects will take the EQ-5D questionnaire",
    "When will the subjects in cohort 3, cohort 4, cohort 5 and cohort 6 will take the EQ-5D questionnaire?",
    "What is the power of the study for the test of efficacy",
    "When will the subjects in cohort 3, cohort 4, cohort 5 and cohort 6 have lumbar punctures for collection of cerebro spinal fluid?"
]

generated_answers = []

for question in Questions:
    print (f"Question: {question}")
    answer = chain.invoke({'question': question})
    generated_answers.append(answer)
    print (f"Answer: {answer}")
    print ()

Question: How many phases is the zuma 1 study separated into ?
Answer: Based on the document provided, the Phase 1 Study of axicabtagene ciloleucel is separated into three phases:

1. Screening phase
2. Dose-limiting toxicity (DLT) evaluable phase
3. Study duration phase

Therefore, the Zuma 1 study is separated into 3 phases.

Question: How many cohorts are there in the zuma 1 study ?
Answer: Based on the provided document, there are 6 cohorts in the Phase 1 Study of KTE-C19:

1. Cohort 1
2. Cohort 2
3. Cohort 3
4. Cohort 4
5. Cohort 5
6. Cohort 6

Each cohort has a specific dose level of axicabtagene ciloleucel, and the study is designed to evaluate the safety and efficacy of the drug at different dose levels.

Question: What periods in the zuma 1 study does each subject proceed through ?
Answer: Based on the provided document, here are the periods that each subject will proceed through in the ZUMA-1 study:

1. Screening period: This starts when the subject signs the informed consent

### Performance matrix


In [11]:
# ground truth answers
answers = [
    "The ZUMA-1 study is separated into three distinct phases: Phase 1 study, Phase 2 pivotal study (Cohort 1 and Cohort 2), and Phase 2 safety management study (Cohort 3, Cohort 4, Cohort 5, and Cohort 6).",
    "In the ZUMA-1 study, there are six cohorts mentioned in the provided context: Cohorts 1 and 2 in the Phase 2 pivotal study Cohort 3 in the Phase 2 safety management study Cohort 4 Cohort 5 Cohort 6 So, there are a total of six cohorts in the ZUMA-1 study.",
    "Each subject in the ZUMA-1 study proceeds through the following study periods: Screening period, Enrollment/Leukapheresis period, Bridging therapy (if applicable; safety management study only) or debulking therapy (if applicable, safety management study, Cohort 5 only), Conditioning chemotherapy period, Investigational product (IP) treatment period, Post-treatment assessment period, Long-term follow-up period. These periods are followed in sequence by each subject in the study.",
    "The primary objective of the Phase 1 ZUMA-1 study is to evaluate the safety of axicabtagene ciloleucel regimens.",
    "The secondary objectives of the Phase 1 study include: Objective response rate (complete response [CR] + partial response [PR]) per the revised International Working Group (IWG) Response Criteria for Malignant Lymphoma. Duration of response (DOR), Overall survival (OS), Progression-free survival (PFS), Incidence of adverse events and clinically significant changes in safety lab values, Levels of anti-CD19 CAR T cells in blood",
    "Levels of cytokines in serum. Incidence of anti-axicabtagene ciloleucel antibodies",
    "The primary objective of the Phase 2 pivotal study is to evaluate the efficacy of axicabtagene ciloleucel, as measured by the objective response rate (ORR) in subjects with diffuse large B-cell lymphoma (DLBCL), primary mediastinal B-cell lymphoma (PMBCL), and transformed follicular lymphoma (TFL).",
    "The secondary objectives of the Phase 2 pivotal study include assessing the safety and tolerability of axicabtagene ciloleucel, additional efficacy endpoints such as DOR, PFS, OS, levels of anti-CD19 CAR T cells in blood, levels of cytokines in serum, and the incidence of anti-axicabtagene ciloleucel antibodies.",
    "The primary objectives of the Phase 2 safety management study are to assess the impact of prophylactic regimens, earlier interventions, tumor debulking, and prophylactic steroid use on the rate and severity of cytokine release syndrome (CRS) and neurologic toxicity.",
    "The secondary objectives of the Phase 2 safety management study include assessing efficacy endpoints (ORR, DOR, PFS, OS), levels of anti-CD19 chimeric antigen receptor (CAR) T cells, cytokines in blood/serum, and the change in European Quality of Life-5 Dimensions (EQ-5D) scores from baseline to Month 6.",
    "The primary endpoint of the Phase 1 study is the incidence of DLT (dose-limiting toxicities)",
    "The secondary endpoints of the Phase 1 study include: Objective response rate (complete response [CR] + partial response [PR]) per the revised International Working Group (IWG) Response Criteria for Malignant Lymphoma. Duration of response (DOR). Overall survival (OS). Progression-free survival (PFS). Incidence of adverse events and clinically significant changes in safety lab values. Levels of anti-CD19 CAR T cells in blood. Levels of cytokines in serum. Incidence of anti-axicabtagene ciloleucel antibodies.",
    "The primary endpoint for the Phase 2 pivotal study (Cohort 1 and Cohort 2) of the study is objective response rate per the revised IWG Response Criteria for Malignant Lymphoma (Cheson et al, 2007) as determined by the study investigators. This endpoint will be based on a modified intent-to-treat (mITT) population consisting of all subjects enrolled and treated with axicabtagene ciloleucel at a dose of at least 1 X 106 anti-CD19 CAR T cells/kg. This study uses a single-arm design to test for an improvement in response",
    "The secondary endpoints of the Phase 2 pivotal study include: Objective response rate per Independent Radiology Review Committee (IRRC). Duration of response (DOR). Progression-free Survival (PFS). Overall Survival (OS). Incidence of adverse events and clinical significant changes in safety lab values, Levels of anti-CD19 CAR T cells in blood, Levels of cytokines in serum. Incidence of anti-axicabtagene ciloleucel antibodies. These secondary endpoints are important for assessing the safety, efficacy, and other factors related to the treatment with axicabtagene ciloleucel in subjects with DLBCL, PMBCL, and TFL.",
    "Primary endpoint of phase 2 safety management study is Incidence and severity of CRS and neurologic toxicities",
    "The secondary endpoint of the Phase 2 Safety Management Study includes the assessment of efficacy endpoints such as Objective Response Rate (ORR), Duration of Response (DOR), Progression-Free Survival (PFS), Overall Survival (OS), levels of anti-CD19 CAR T cells in blood, levels of cytokines in blood, and changes over time in the EQ-5D scale score and EQ-5D VAS score for subjects treated in Cohort 3, Cohort 4, Cohort 5, and Cohort 6.",
    "Approximately 268 to 286 subjects will be enrolled and dosed in the ZUMA-1 study. The breakdown of subjects across different phases and cohorts is as follows: Phase 1 Study: Approximately 6 to 24 subjects. Phase 2 Pivotal Study: Approximately 92 subjects enrolled into 2 cohorts (Cohort 1: approximately 72 subjects, Cohort 2: approximately 20 subjects). Phase 2 Safety Management Study: Approximately 170 subjects enrolled and dosed within 4 cohorts (Cohort 3: approximately 40 subjects, Cohort 4: approximately 40 subjects, Cohort 5: approximately 50 subjects, Cohort 6: approximately 40 subjects)",
    "In the Phase 1 study of the ZUMA-1 study, approximately 6 to 24 subjects will be enrolled.",
    "In the Phase 2 pivotal study of ZUMA-1, approximately 92 subjects will be enrolled into 2 separate cohorts. Cohort 1 will enroll approximately 72 adult subjects with refractory DLBCL, and Cohort 2 will enroll approximately 20 adult subjects with refractory PMBCL and TFL.",
    "Approximately 72 adult subjects with refractory DLBCL will be enrolled in Cohort 1 of the Phase 2 pivotal study of ZUMA-1.",
    "Approximately 20 adult subjects with refractory PMBCL and TFL will be enrolled in Cohort 2 of the Phase 2 pivotal study of ZUMA-1",
    "Approximately 170 subjects will be enrolled and dosed within 4 cohorts in the Phase 2 Safety Management Study. Cohort 3 will have approximately 40 subjects, Cohort 4 will have approximately 40 subjects, Cohort 5 will have approximately 50 subjects, and Cohort 6 will have approximately 40 subjects.",
    "Approximately 40 subjects will be enrolled in Cohort 3 of the Phase 2 Safety Management Study",
    "Approximately 40 subjects will be enrolled and dosed in Cohort 4 of the Phase 2 Safety Management Study.",
    "Approximately 50 subjects will be enrolled and dosed in Cohort 5 of the Phase 2 Safety Management Study.",
    "Approximately 40 subjects will be enrolled and dosed in Cohort 6 of the Phase 2 Safety Management Study.",
    "The investigational product in this study is Axicabtagene ciloleucel, which consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg.",
    "The investigational product treatment consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg. For subjects weighing more than 100 kg, a maximum flat dose of 2 x 10.8 anti-CD19 CAR T cells will be administered. If subjects initially respond and subsequently relapse, they may be eligible for a second course of conditioning chemotherapy and axicabtagene ciloleucel",
    "Axicabtagene ciloleucel treatment consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg. For subjects weighing greater than 100 kg, a maximum flat dose of 2 x 10.8 anti-CD19 CAR T cells will be administered. In cases of initial response followed by relapse, subjects may be eligible for a second course of conditioning chemotherapy and axicabtagene ciloleucel",
    "Bridging therapy may be considered for subjects with high disease burden at screening or baseline assessments, such as bulky disease or rapidly progressing disease. The decision to use bridging therapy is at the discretion of the investigator.",
    "Debulking therapy should be considered after leukapheresis and completed at least 14 days prior to initiating conditioning chemotherapy or axicabtagene ciloleucel administration. It is aimed at reducing lymphoma burden.",
    "Subjects enrolled in the Phase 2 Safety Management Study, specifically those in Cohort 5, may be considered for debulking therapy. Debulking therapy is aimed at reducing lymphoma burden and should be administered after leukapheresis/enrollment and completed at least 14 days prior to the start of conditioning chemotherapy or axicabtagene ciloleucel administration. The specific debulking therapy regimens are outlined in Table 3, and other debulking treatment options may be considered in select cases after discussion with the Kite medical monitor.",
    "The conditioning chemotherapy regimen used before administering axicabtagene ciloleucel consists of fludarabine 30 mg/m2/day and cyclophosphamide 500 mg/m2/day, administered for 3 days.",
    "The dose of the conditioning chemotherapy used in the study is cyclophosphamide 500 mg/m2 given intravenously over approximately 60 minutes on Day -5, Day -4, and Day -3, along with fludarabine 30 mg/m2 given intravenously over approximately 30 minutes on the same days.",
    "The conditioning chemotherapy regimen for this study consists of fludarabine and cyclophosphamide. Fludarabine is administered at 30 mg/m2/day, and cyclophosphamide is administered at 500 mg/m2/day. The regimen is given for 3 days in a row. Additional details about the administration and hydration requirements are provided in the study protocol.",
    "The fludarabine dose used in the conditioning chemotherapy regimen is 30 mg/m2/day.",
    "The cyclophosphamide dose used in the conditioning chemotherapy regimen is 500 mg/m2/day, administered for 3 concurrent days.",
    "The conditioning chemotherapy regimen is given for either 5 days or 7 days, depending on the specific cohort and treatment plan.",
    "Subjects assigned to Cohort 3 in the Phase 2 Safety Management Study will receive conditioning chemotherapy and axicabtagene ciloleucel as described above. Additionally, they will receive prophylactic tocilizumab and levetiracetam for toxicity management as outlined in the study details.",
    "Subjects assigned to Cohort 4 in the Phase 2 safety management study will receive the same conditioning chemotherapy and axicabtagene ciloleucel regimen as described above. Additionally, toxicity management will intervene at lower grades of CRS and neurologic toxicities as outlined in Section 6.4.1.",
    "Subjects assigned to Cohort 5 in the Phase 2 Safety Management Study will receive a non-myeloablative conditioning regimen consisting of cyclophosphamide and fludarabine to induce lymphocyte depletion. The conditioning chemotherapy regimen will be initiated with cyclophosphamide and fludarabine beginning on Day -5 through Day -1.",
    "Subjects in Cohort 5 of the Phase 2 Safety Management Study will receive debulking therapy to reduce lymphoma burden. The debulking therapy options for Cohort 5 are outlined in Table 3 of the study protocol. The debulking therapy should be administered after leukapheresis and before the conditioning chemotherapy or axicabtagene ciloleucel treatment.",
    "Subjects assigned to Cohort 6 in the Phase 2 Safety Management Study will receive the same conditioning chemotherapy regimen as described above, which consists of cyclophosphamide and fludarabine. The conditioning chemotherapy regimen will be initiated with cyclophosphamide and fludarabine beginning on Day -5 through Day -1.",
    "At specific time points as outlined in the schedule of assessments, subjects will undergo the following assessments/procedures: collection of informed consent, general medical history including previous treatments for NHL, physical exam including vital signs and performance status, neurologic assessments, blood draws for complete blood count (CBC), chemistry panels, cytokines, C-reactive protein, lymphocyte subsets, antiaxicabtagene ciloleucel antibodies, replication-competent retrovirus (RCR) and anti-CD19 CAR T cell analysis. Women of childbearing potential will undergo a urine or serum pregnancy test.",
    "At baseline, the subjects will undergo the following procedures: Collection of informed consent, General medical history including previous treatments for NHL, Physical exam including vital signs and performance status, Neurologic assessments. Blood draws for complete blood count (CBC), chemistry panels, cytokines, C-reactive protein, lymphocyte subsets, anti-axicabtagene ciloleucel antibodies, replication-competent retrovirus (RCR), and anti-CD19 CAR T cell analysis. Women of childbearing potential will undergo a urine or serum pregnancy test, Baseline electrocardiogram (ECG), Echocardiogram (ECHO), Brain magnetic resonance image (MRI), Positron emission tomography, computed tomography (PET-CT), Possible bone marrow aspirate/biopsy, Leukapheresis. Additionally, subjects assigned to specific cohorts will complete the EQ-5D questionnaire at baseline and have lumbar punctures performed for the collection of cerebrospinal fluid (CSF) before and after axicabtagene ciloleucel infusion.",
    "Subjects enrolled in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire.",
    "Subjects in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire at the following time points: The EQ-5D questionnaire will be completed by the subject at the screening visit, The EQ-5D questionnaire will be administered prior to any other assessments/procedures being performed at the screening visit, The EQ-5D questionnaire will be administered at Week 4 (± 3 days), The EQ-5D questionnaire will be administered at Month 3 (± 1 week), In the Long-term Follow-up Period, the EQ-5D questionnaire will be administered at Month 6. These are the specific time points when subjects in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire",
    "The study has a power of ≥ 90% to distinguish between an active therapy with a 40% true response rate from a therapy with a response rate of 20% or less with a 1-sided alpha level of 0.025.",
    "Subjects in cohort 3, cohort 4, cohort 5, and cohort 6 will have lumbar punctures for the collection of cerebrospinal fluid (CSF) samples at the following time points: After eligibility is confirmed and prior to the start of conditioning chemotherapy, Post axicabtagene ciloleucel infusion on Day 5 (±3 days), At the Week 4 visit (±3 days)."
]


In [15]:
ground_truth_answers = [
    """The ZUMA-1 study is separated into three distinct phases: Phase 1 study, Phase 2 pivotal study (Cohort 1 and Cohort 2), and Phase 2 safety management study (Cohort 3, Cohort 4, Cohort 5, and Cohort 6).""",
    """In the ZUMA-1 study, there are six cohorts mentioned in the provided context:

    Cohorts 1 and 2 in the Phase 2 pivotal study

    Cohort 3 in the Phase 2 safety management study

    Cohort 4

    Cohort 5

    Cohort 6

    So, there are a total of six cohorts in the ZUMA-1 study.""",
    """Each subject in the ZUMA-1 study proceeds through the following study periods:

    Screening period

    Enrollment/Leukapheresis period

    Bridging therapy (if applicable; safety management study only) or debulking therapy (if applicable, safety management study, Cohort 5 only)

    Conditioning chemotherapy period

    Investigational product (IP) treatment period

    Post-treatment assessment period

    Long-term follow-up period

    These periods are followed in sequence by each subject in the study.""",
    """The primary objective of the Phase 1 ZUMA-1 study is to evaluate the safety of axicabtagene ciloleucel regimens.""",
    """The secondary objectives of the Phase 1 study include:

    Objective response rate (complete response [CR] + partial response [PR]) per the revised International Working Group (IWG) Response Criteria for Malignant Lymphoma

    Duration of response (DOR)

    Overall survival (OS)

    Progression-free survival (PFS)

    Incidence of adverse events and clinically significant changes in safety lab values

    Levels of anti-CD19 CAR T cells in blood

    Levels of cytokines in serum

    Incidence of anti-axicabtagene ciloleucel antibodies""",
    """The primary objective of the Phase 2 pivotal study is to evaluate the efficacy of axicabtagene ciloleucel, as measured by the objective response rate (ORR) in subjects with diffuse large B-cell lymphoma (DLBCL), primary mediastinal B-cell lymphoma (PMBCL), and transformed follicular lymphoma (TFL).""",
    """The secondary objectives of the Phase 2 pivotal study include assessing the safety and tolerability of axicabtagene ciloleucel, additional efficacy endpoints such as DOR, PFS, OS, levels of anti-CD19 CAR T cells in blood, levels of cytokines in serum, and the incidence of anti-axicabtagene ciloleucel antibodies.""",
    """The primary objectives of the Phase 2 safety management study are to assess the impact of prophylactic regimens, earlier interventions, tumor debulking, and prophylactic steroid use on the rate and severity of cytokine release syndrome (CRS) and neurologic toxicity.""",
    """The secondary objectives of the Phase 2 safety management study include assessing efficacy endpoints (ORR, DOR, PFS, OS), levels of anti-CD19 chimeric antigen receptor (CAR) T cells, cytokines in blood/serum, and the change in European Quality of Life-5 Dimensions (EQ-5D) scores from baseline to Month 6.""",
    """The primary endpoint of the Phase 1 study is the incidence of DLT (dose-limiting toxicities)""",
    """The secondary endpoints of the Phase 1 study include:
    Objective response rate (complete response [CR] + partial response [PR]) per the revised International Working Group (IWG) Response Criteria for Malignant Lymphoma.
    Duration of response (DOR).
    Overall survival (OS).
    Progression-free survival (PFS).
    Incidence of adverse events and clinically significant changes in safety lab values.
    Levels of anti-CD19 CAR T cells in blood.
    Levels of cytokines in serum.
    Incidence of anti-axicabtagene ciloleucel antibodies.""",
    """The primary endpoint for the Phase 2 pivotal study (Cohort 1 and Cohort 2) of the study is objective response rate per the revised IWG Response Criteria for Malignant Lymphoma (Cheson et al, 2007) as determined by the study investigators. This endpoint will be based on a modified intent-to-treat (mITT) population consisting of all subjects enrolled and treated with axicabtagene ciloleucel at a dose of at least 1 X 106 anti-CD19 CAR T cells/kg. This study uses a single-arm design to test for an improvement in response""",
    """The secondary endpoints of the Phase 2 pivotal study include:

    Objective response rate per Independent Radiology Review Committee (IRRC)

    Duration of response (DOR)

    Progression-free Survival (PFS)

    Overall Survival (OS)

    Incidence of adverse events and clinical significant changes in safety lab values

    Levels of anti-CD19 CAR T cells in blood

    Levels of cytokines in serum

    Incidence of anti-axicabtagene ciloleucel antibodies

    These secondary endpoints are important for assessing the safety, efficacy, and other factors related to the treatment with axicabtagene ciloleucel in subjects with DLBCL, PMBCL, and TFL.""",
    """Primary endpoint of phase 2 safety management study is Incidence and severity of CRS and neurologic toxicities""",
    """The secondary endpoint of the Phase 2 Safety Management Study includes the assessment of efficacy endpoints such as Objective Response Rate (ORR), Duration of Response (DOR), Progression-Free Survival (PFS), Overall Survival (OS), levels of anti-CD19 CAR T cells in blood, levels of cytokines in blood, and changes over time in the EQ-5D scale score and EQ-5D VAS score for subjects treated in Cohort 3, Cohort 4, Cohort 5, and Cohort 6.""",
    """Approximately 268 to 286 subjects will be enrolled and dosed in the ZUMA-1 study. The breakdown of subjects across different phases and cohorts is as follows:

    Phase 1 Study: Approximately 6 to 24 subjects

    Phase 2 Pivotal Study: Approximately 92 subjects enrolled into 2 cohorts (Cohort 1: approximately 72 subjects, Cohort 2: approximately 20 subjects)

    Phase 2 Safety Management Study: Approximately 170 subjects enrolled and dosed within 4 cohorts (Cohort 3: approximately 40 subjects, Cohort 4: approximately 40 subjects, Cohort 5: approximately 50 subjects, Cohort 6: approximately 40 subjects)""",
    """In the Phase 1 study of the ZUMA-1 study, approximately 6 to 24 subjects will be enrolled.""",
    """In the Phase 2 pivotal study of ZUMA-1, approximately 92 subjects will be enrolled into 2 separate cohorts. Cohort 1 will enroll approximately 72 adult subjects with refractory DLBCL, and Cohort 2 will enroll approximately 20 adult subjects with refractory PMBCL and TFL.""",
    """Approximately 72 adult subjects with refractory DLBCL will be enrolled in Cohort 1 of the Phase 2 pivotal study of ZUMA-1.""",
    """Approximately 20 adult subjects with refractory PMBCL and TFL will be enrolled in Cohort 2 of the Phase 2 pivotal study of ZUMA-1""",
    """Approximately 170 subjects will be enrolled and dosed within 4 cohorts in the Phase 2 Safety Management Study. Cohort 3 will have approximately 40 subjects, Cohort 4 will have approximately 40 subjects, Cohort 5 will have approximately 50 subjects, and Cohort 6 will have approximately 40 subjects.""",
    """Approximately 40 subjects will be enrolled in Cohort 3 of the Phase 2 Safety Management Study""",
    """Approximately 40 subjects will be enrolled and dosed in Cohort 4 of the Phase 2 Safety Management Study.""",
    """Approximately 50 subjects will be enrolled and dosed in Cohort 5 of the Phase 2 Safety Management Study.""",
    """Approximately 40 subjects will be enrolled and dosed in Cohort 6 of the Phase 2 Safety Management Study.""",
    """The investigational product in this study is Axicabtagene ciloleucel, which consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg.""",
    """The investigational product treatment consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg. For subjects weighing more than 100 kg, a maximum flat dose of 2 x 10.8 anti-CD19 CAR T cells will be administered. If subjects initially respond and subsequently relapse, they may be eligible for a second course of conditioning chemotherapy and axicabtagene ciloleucel""",
    """Axicabtagene ciloleucel treatment consists of a single infusion of CAR transduced autologous T cells administered intravenously at a target dose of 2 x 10.6 anti-CD19 CAR T cells/kg. For subjects weighing greater than 100 kg, a maximum flat dose of 2 x 10.8 anti-CD19 CAR T cells will be administered. In cases of initial response followed by relapse, subjects may be eligible for a second course of conditioning chemotherapy and axicabtagene ciloleucel""",
    """Bridging therapy may be considered for subjects with high disease burden at screening or baseline assessments, such as bulky disease or rapidly progressing disease. The decision to use bridging therapy is at the discretion of the investigator.""",
    """Debulking therapy should be considered after leukapheresis and completed at least 14 days prior to initiating conditioning chemotherapy or axicabtagene ciloleucel administration. It is aimed at reducing lymphoma burden.""",
    """Subjects enrolled in the Phase 2 Safety Management Study, specifically those in Cohort 5, may be considered for debulking therapy. Debulking therapy is aimed at reducing lymphoma burden and should be administered after leukapheresis/enrollment and completed at least 14 days prior to the start of conditioning chemotherapy or axicabtagene ciloleucel administration. The specific debulking therapy regimens are outlined in Table 3, and other debulking treatment options may be considered in select cases after discussion with the Kite medical monitor.""",
    """The conditioning chemotherapy regimen used before administering axicabtagene ciloleucel consists of fludarabine 30 mg/m2/day and cyclophosphamide 500 mg/m2/day, administered for 3 days.""",
    """The dose of the conditioning chemotherapy used in the study is cyclophosphamide 500 mg/m2 given intravenously over approximately 60 minutes on Day -5, Day -4, and Day -3, along with fludarabine 30 mg/m2 given intravenously over approximately 30 minutes on the same days.""",
    """The conditioning chemotherapy regimen for this study consists of fludarabine and cyclophosphamide. Fludarabine is administered at 30 mg/m2/day, and cyclophosphamide is administered at 500 mg/m2/day. The regimen is given for 3 days in a row. Additional details about the administration and hydration requirements are provided in the study protocol.""",
    """The fludarabine dose used in the conditioning chemotherapy regimen is 30 mg/m2/day.""",
    """The cyclophosphamide dose used in the conditioning chemotherapy regimen is 500 mg/m2/day, administered for 3 concurrent days.""",
    """The conditioning chemotherapy regimen is given for either 5 days or 7 days, depending on the specific cohort and treatment plan.""",
    """Subjects assigned to Cohort 3 in the Phase 2 Safety Management Study will receive conditioning chemotherapy and axicabtagene ciloleucel as described above. Additionally, they will receive prophylactic tocilizumab and levetiracetam for toxicity management as outlined in the study details.""",
    """Subjects assigned to Cohort 4 in the Phase 2 safety management study will receive the same conditioning chemotherapy and axicabtagene ciloleucel regimen as described above. Additionally, toxicity management will intervene at lower grades of CRS and neurologic toxicities as outlined in Section 6.4.1.""",
    """Subjects assigned to Cohort 5 in the Phase 2 Safety Management Study will receive a non-myeloablative conditioning regimen consisting of cyclophosphamide and fludarabine to induce lymphocyte depletion. The conditioning chemotherapy regimen will be initiated with cyclophosphamide and fludarabine beginning on Day -5 through Day -1.""",
    """Subjects in Cohort 5 of the Phase 2 Safety Management Study will receive debulking therapy to reduce lymphoma burden. The debulking therapy options for Cohort 5 are outlined in Table 3 of the study protocol. The debulking therapy should be administered after leukapheresis and before the conditioning chemotherapy or axicabtagene ciloleucel treatment.""",
    """Subjects assigned to Cohort 6 in the Phase 2 Safety Management Study will receive the same conditioning chemotherapy regimen as described above, which consists of cyclophosphamide and fludarabine. The conditioning chemotherapy regimen will be initiated with cyclophosphamide and fludarabine beginning on Day -5 through Day -1.""",
    """At specific time points as outlined in the schedule of assessments, subjects will undergo the following assessments/procedures: collection of informed consent, general medical history including previous treatments for NHL, physical exam including vital signs and performance status, neurologic assessments, blood draws for complete blood count (CBC), chemistry panels, cytokines, C-reactive protein, lymphocyte subsets, antiaxicabtagene ciloleucel antibodies, replication-competent retrovirus (RCR) and anti-CD19 CAR T cell analysis. Women of childbearing potential will undergo a urine or serum pregnancy test.""",
    """At baseline, the subjects will undergo the following procedures:

    Collection of informed consent

    General medical history including previous treatments for NHL

    Physical exam including vital signs and performance status

    Neurologic assessments

    Blood draws for complete blood count (CBC), chemistry panels, cytokines, C-reactive protein, lymphocyte subsets, anti-axicabtagene ciloleucel antibodies, replication-competent retrovirus (RCR), and anti-CD19 CAR T cell analysis

    Women of childbearing potential will undergo a urine or serum pregnancy test

    Baseline electrocardiogram (ECG)

    Echocardiogram (ECHO)

    Brain magnetic resonance image (MRI)

    Positron emission tomography–computed tomography (PET-CT)

    Possible bone marrow aspirate/biopsy

    Leukapheresis

    Additionally, subjects assigned to specific cohorts will complete the EQ-5D questionnaire at baseline and have lumbar punctures performed for the collection of cerebrospinal fluid (CSF) before and after axicabtagene ciloleucel infusion.""",
    """Subjects enrolled in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire.""",
    """Subjects in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire at the following time points:

    The EQ-5D questionnaire will be completed by the subject at the screening visit.

    The EQ-5D questionnaire will be administered prior to any other assessments/procedures being performed at the screening visit.

    The EQ-5D questionnaire will be administered at Week 4 (± 3 days).

    The EQ-5D questionnaire will be administered at Month 3 (± 1 week).

    In the Long-term Follow-up Period, the EQ-5D questionnaire will be administered at Month 6.

    These are the specific time points when subjects in Cohort 3, Cohort 4, Cohort 5, and Cohort 6 will take the EQ-5D questionnaire""",
    """The study has a power of ≥ 90% to distinguish between an active therapy with a 40% true response rate from a therapy with a response rate of 20% or less with a 1-sided alpha level of 0.025.""",
    """Subjects in cohort 3, cohort 4, cohort 5, and cohort 6 will have lumbar punctures for the collection of cerebrospinal fluid (CSF) samples at the following time points:

    After eligibility is confirmed and prior to the start of conditioning chemotherapy.

    Post axicabtagene ciloleucel infusion on Day 5 (±3 days).

    At the Week 4 visit (±3 days)."""
]

In [16]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from difflib import SequenceMatcher

# Download necessary NLTK data
nltk.download('punkt')

def preprocess_paragraph(paragraph):
    """Converts a paragraph into a list of sentences, each sentence being a list of words."""
    sentences = nltk.sent_tokenize(paragraph)
    tokenized_sentences = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]
    return tokenized_sentences

def calculate_exact_match(predicted, ground_truth):
    return int(predicted.strip().lower() == ground_truth.strip().lower())

def calculate_bleu(predicted, ground_truth):
    predicted_sentences = preprocess_paragraph(predicted)
    ground_truth_sentences = preprocess_paragraph(ground_truth)
    
    # Flatten the lists of sentences
    predicted_tokens = [token for sentence in predicted_sentences for token in sentence]
    ground_truth_tokens = [token for sentence in ground_truth_sentences for token in sentence]
    
    # Calculate BLEU score with smoothing
    smoothing_function = SmoothingFunction().method4
    return sentence_bleu([ground_truth_tokens], predicted_tokens, smoothing_function=smoothing_function)

def calculate_similarity(a, b):
    """Calculates a similarity score between two strings using SequenceMatcher."""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()


# Calculate metrics
exact_matches = []
bleu_scores = []
similarity_scores = []

for pred, gt in zip(generated_answers, ground_truth_answers):
    exact_matches.append(calculate_exact_match(pred, gt))
    bleu_scores.append(calculate_bleu(pred, gt))
    similarity_scores.append(calculate_similarity(pred, gt))

# Aggregate results
exact_match_score = sum(exact_matches) / len(exact_matches)
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
average_similarity_score = sum(similarity_scores) / len(similarity_scores)

print(f"Exact Match Score: {exact_match_score}")
print(f"Average BLEU Score: {average_bleu_score}")
print(f"Average Similarity Score: {average_similarity_score}")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UDIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Exact Match Score: 0.0
Average BLEU Score: 0.11888674823326019
Average Similarity Score: 0.16493019232123543


In [17]:
print (bleu_scores)

[0.12899634702914786, 0.06385716751180487, 0.02779306455244843, 0.2235267829375094, 0.07815061923690277, 0.21953426767815928, 0.10973053480193572, 0.5544997954319238, 0.05385099484126001, 0.02116966743398892, 0.06747625776854715, 0.07760831356984997, 0.06268227369902159, 0.04961025956014616, 0.03639940458958957, 0.08089773015784754, 0.046980264321105726, 0.10749710606179641, 0.1395348939244277, 0.0806167042100049, 0.17002517076160525, 0.27829864388162245, 0.39985976557646163, 0.09578585725940554, 0.3867316537771211, 0.16978850387234024, 0.06077006624361228, 0.07012293393436025, 0.039107651592301215, 0.01958255366477119, 0.09167627028417394, 0.03069788927736604, 0.06583464674277996, 0.04130992667081113, 0.05756616439894722, 0.029979926642525415, 0.04054366381894934, 0.19653097672446057, 0.25042585937459083, 0.09118073644093938, 0.11884869744588841, 0.0954303401128398, 0.007798565287447453, 0.09457784265261983, 0.251696695878184, 0.22412561877610537, 0.017996127296059473, 0.0798587174907

## Creating effective vector store for tabular data in pdf


In [5]:
#Extracting tables in the pdf

import camelot

def extract_tables_from_pdf(pdf_path):
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
    extracted_tables = [table.df for table in tables]
    return extracted_tables

tables = extract_tables_from_pdf("protocol.pdf")

c:\Users\UDIT\anaconda3\envs\miimansa\lib\site-packages\camelot\parsers\stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


In [6]:
print (len(tables))

131


In [8]:
# Process tables into embeddings
from langchain_community.embeddings import OllamaEmbeddings
# from sentence_transformers import SentenceTransformer

# table_embedding_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
embeddings_model = OllamaEmbeddings()
text_contents = [page.page_content for page in pages]
# text_embeddings = [embeddings_model.embed_documents(text) for text in text_contents]

def process_table(table):
    table_str = table.to_string(index=False)
    return embeddings_model.embed_documents(table_str)

table_contents = [table.to_string(index=False) for table in tables]
# table_embeddings = [process_table(table) for table in tables]

In [9]:
#Combine text and table data

from langchain.vectorstores import DocArrayInMemorySearch

from langchain_community.document_loaders import PyPDFLoader

# Load and split PDF text data
loader = PyPDFLoader("protocol.pdf")
pages = loader.load_and_split()

# Get text embeddings
text_embeddings = OllamaEmbeddings()
# text_vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=text_embeddings)
# table_vectorstore = DocArrayInMemorySearch.from_texts([table.to_string(index=False) for table in tables], embedding=text_embeddings)
text_vectorstore = DocArrayInMemorySearch.from_texts(text_contents, embedding=embeddings_model)
# Create table vector store
table_vectorstore = DocArrayInMemorySearch.from_texts(table_contents, embedding=embeddings_model)




c:\Users\UDIT\anaconda3\envs\miimansa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class CombinedRetriever:
    def __init__(self, text_store, table_store):
        self.text_store = text_store
        self.table_store = table_store

    def retrieve(self, query, k=5):
        text_results = self.text_store.similarity_search(query, k=k)
        table_results = self.table_store.similarity_search(query, k=k)
        combined_results = text_results + table_results
        # Optionally sort combined_results by relevance score
        return combined_results

# Create the retriever
combined_retriever = CombinedRetriever(text_vectorstore, table_vectorstore)


In [11]:
print (type(combined_retriever))

<class '__main__.CombinedRetriever'>


In [15]:
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_community.models import Ollama

prompt_template = PromptTemplate(
    template="Answer the question based on the given text and table data:\n{context}\nQuestion: {question}\nAnswer:",
    input_variables={"context", "question"}
)

qa_chain = RetrievalQA(
    retriever=combined_retriever,
    model=Ollama(),
    prompt_template=prompt_template
)

ModuleNotFoundError: No module named 'langchain_community.models'